In [1]:
import numpy
import pandas as pd

In [11]:
dat = [x.split() for x in open("data/adventday2.txt").readlines()]
df = pd.DataFrame(dat)
df.columns = ['times', 'letter', 'password']

The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In [3]:
df[['times_lb', 'times_ub']] = df['times'].str.split("-", expand = True)
df['times_lb'] = pd.to_numeric(df['times_lb'])
df['times_ub'] = pd.to_numeric(df['times_ub'])

df['letter'] = df['letter'].str.replace(":", "")

In [4]:
def count_letter(password, letter):
    return len([ch for ch in password if ch == letter])

In [5]:
df['count'] = df.apply(lambda x: count_letter(x['password'], x['letter']), axis=1)
df['valid'] = (df['count'] <= df['times_ub']) & (df['count'] >= df['times_lb'])

In [6]:
df['valid'].value_counts()

True     638
False    362
Name: valid, dtype: int64

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

In [7]:
df = df[['password', 'letter', 'times_lb', 'times_ub']]
df = df.rename(columns = {'times_lb': 'pos1',
                  'times_ub': 'pos2'})

In [8]:
def loc_letter(password, letter, pos1, pos2):
    return (password[pos1 - 1] == letter) + (password[pos2 - 1] == letter)

In [9]:
df['count'] = df.apply(lambda x: loc_letter(x['password'], x['letter'], x['pos1'], x['pos2']), axis=1)
df['valid'] = False
df.loc[df['count'] == 1, 'valid'] = True

In [10]:
df['valid'].value_counts()

True     699
False    301
Name: valid, dtype: int64